In [ ]:
import numpy as np
import pandas as pd
import string
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Creating a Data List and Data Frame from Text File

In [ ]:
f=open('/content/SMSSpamCollection.txt' , 'r')
datalist=[]
glue=' '

for i in f:
    item=i.split()
    label=item[0]
    msg=glue.join(item[1::])

    datalist.append([label,msg])


df=pd.DataFrame(datalist,columns=['Label','Msg'])


In [ ]:
df.head()

,Label,Msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
datalist[0:2]

[['ham',
  'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
 ['ham', 'Ok lar... Joking wif u oni...']]

Pre Processing Data


1.   Removing punctuation and special characters
2.   Lower Casing
3.   Removing Stopwords
4.   Stem Words Only







In [ ]:
def conditioning(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text=text.lower()
    res=[]
    for i in text.split():
      if i not in stopwords.words('english'):
        res.append(ps.stem(i))
    return res

In [ ]:
conditioning('I Loved the Lectures on Differential Learning 1 @')


['love', 'lectur', 'differenti', 'learn', '1']

Building a Multinomial NB using Bag of Words

In [ ]:

vocab={}
spamcount=0
hamcount=0

Nspam=0
Nham=0

spam_dict={} #Parameter 1
ham_dict={} #Parameter 2




In [ ]:
for i in datalist:
    if i[0]=='spam':
        spamcount+=1
        i=conditioning(i[1])

        for word in i:
            Nspam+=1
            if word in spam_dict:
                spam_dict[word]+=1
            else:
                spam_dict[word]=1



    else:
        hamcount+=1
        i=conditioning(i[1])

        for word in i:
            Nham+=1
            if word in ham_dict:
                ham_dict[word]+=1
            else:
                ham_dict[word]=1




In [ ]:
s1=set(spam_dict.keys())
s2=set(ham_dict.keys())
vocab=s1.union(s2)
Nvocab=len(vocab)

In [ ]:



Pspam=spamcount/(spamcount+hamcount) #Parameter 4
Pham=1-Pspam                         #Parameter 5

# print(spam_dict)
# print()
# print()
# print(ham_dict)
# print()
# print()
# print(Nvocab)
# print(Nspam)
# print(Nham)

P_w_spam={}     #Parameter 5
P_w_ham={}      #Parameter 6

for i in spam_dict:
    P_w_spam[i]=(spam_dict[i] + 1)/(Nspam+Nvocab)


for i in ham_dict:
    P_w_ham[i]=(ham_dict[i] + 1)/(Nham+Nvocab)


In [ ]:
# A List of the Trained Parameter
parameters=[spam_dict,ham_dict,Pspam,Pham,P_w_spam,P_w_ham]


Funtion to Predict new Message on the Trained Model

In [ ]:

def predict_message(message):
  message=conditioning(message)
  P_spam_given_w=Pspam
  P_ham_given_w=Pham

  for i in message:
      if i in spam_dict:
          P_spam_given_w*=P_w_spam[i]
      if i in ham_dict:
          P_ham_given_w*=P_w_ham[i]

  if P_spam_given_w > P_ham_given_w:
      print("SPAM")
  else:
      print("HAM")

In [ ]:
message="Car loans available starting at 32.64 now"
predict_message(message)

In [ ]:
with open('model.pkl', 'wb') as x:
        pickle.dump(parameters, x)